In [1]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from warnings import simplefilter

numbers = pd.read_pickle('level1data')
df = numbers.copy()

In /Users/benjaminliu/opt/anaconda3/envs/myenv/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/benjaminliu/opt/anaconda3/envs/myenv/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/benjaminliu/opt/anaconda3/envs/myenv/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/benjaminliu/opt/anaconda3/envs/myenv/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

In [2]:
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

init_notebook_mode(connected=True)

layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(103, 128, 159, .8)')
fig = go.Figure(layout=layout)
fig.update_layout(
    font_color="white",
)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

EMA_12 = pd.Series(df['close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['close'].ewm(span=26, min_periods=26).mean())
df['MACD'] = pd.Series(EMA_12 - EMA_26)
df['MACD_signal'] = pd.Series(df.MACD.ewm(span=9, min_periods=9).mean())

In [3]:
df = df.iloc[33:] # Because of moving averages and MACD line
df = df[:-1]      # Because of shifting close price

df.index = range(len(df))

test_size  = 0.15
valid_size = 0.15

test_split_idx  = int(df.shape[0] * (1-test_size))
valid_split_idx = int(df.shape[0] * (1-(valid_size+test_size)))

train_df  = df.loc[:valid_split_idx].copy()
valid_df  = df.loc[valid_split_idx+1:test_split_idx].copy()
test_df   = df.loc[test_split_idx+1:].copy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=train_df.date, y=train_df.close, name='Training'))
fig.add_trace(go.Scatter(x=valid_df.date, y=valid_df.close, name='Validation'))
fig.add_trace(go.Scatter(x=test_df.date,  y=test_df.close,  name='Test'))
fig.show()

In [4]:
drop_cols = ['date', 'volume', 'open', 'low', 'high', 'symbol', 'id', 'key', 'updated', 'label', 'subkey']

train_df = train_df.drop(drop_cols, 1)
valid_df = valid_df.drop(drop_cols, 1)
test_df  = test_df.drop(drop_cols, 1)

y_train = train_df['close'].copy()
X_train = train_df.drop(['close'], 1)

y_valid = valid_df['close'].copy()
X_valid = valid_df.drop(['close'], 1)

y_test  = test_df['close'].copy()
X_test  = test_df.drop(['close'], 1)

X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   changeOverTime        152 non-null    float64
 1   marketChangeOverTime  152 non-null    float64
 2   uOpen                 152 non-null    float64
 3   uClose                152 non-null    float64
 4   uHigh                 152 non-null    float64
 5   uLow                  152 non-null    float64
 6   uVolume               152 non-null    int64  
 7   fOpen                 152 non-null    float64
 8   fClose                152 non-null    float64
 9   fHigh                 152 non-null    float64
 10  fLow                  152 non-null    float64
 11  fVolume               152 non-null    int64  
 12  change                152 non-null    float64
 13  changePercent         152 non-null    float64
 14  MACD                  152 non-null    float64
 15  MACD_signal           1

In [5]:
parameters = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'max_depth': [8, 10, 12, 15],
    'gamma': [0.001, 0.005, 0.01, 0.02],
    'random_state': [42]
}

eval_set = [(X_train, y_train), (X_valid, y_valid)]
model = xgb.XGBRegressor(eval_set=eval_set, objective='reg:squarederror', verbose = False)
clf = GridSearchCV(model, parameters)

model = xgb.XGBRegressor(eval_set = [(X_train, y_train), (X_valid, y_valid)], objective='reg:squarederror')
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

[14:23:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { eval_set } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1,
             eval_set=[(     changeOverTime  marketChangeOverTime   uOpen  uClose     uHigh    uLow  \
0         -0.228531             -0.228531  146.37  152.72  154.2100  143.63   
1         -0.210497             -0.210497  154.77  156.29  157.3000  153.02   
2         -0.202970             -0.202970  158.50  157.78  162.1200  155.98   
3         -0.202465             -0.202465  160.00  157.88  161.6500  156.85   
4...
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbo

In [6]:
y_pred = model.predict(X_test)
print(f'y_true = {np.array(y_test)[:50]}')
print(f'y_pred = {y_pred[:50]}')

y_true = [193.43 198.28 195.53 196.54 194.13 193.43 188.33 188.78 186.19 189.42
 193.8  194.79 189.62 192.35 198.43 197.56 199.36 198.   199.87 197.29
 193.5  193.62 196.75 197.3  196.81 197.93 197.11 195.17 199.91 199.77
 199.18 195.13]
y_pred = [194.25632 197.21373 195.95807 196.19518 194.94057 192.83977 189.03233
 188.9395  186.45418 190.0212  194.01418 194.3239  190.99844 192.5098
 197.14401 197.22302 197.27045 197.22185 197.0675  196.96082 194.00435
 193.14923 197.33105 197.25246 197.05493 197.41023 196.86545 195.32448
 197.39691 197.243   197.06541 195.31529]


In [7]:
predicted_prices = df.loc[test_split_idx+1:].copy()
predicted_prices['Close'] = y_pred

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=df.date, y=df.close,
                         name='Truth',
                         marker_color='LightSkyBlue'), row=1, col=1)

fig.add_trace(go.Scatter(x=predicted_prices.date,
                         y=predicted_prices.close,
                         name='Prediction',
                         marker_color='MediumPurple'), row=1, col=1)

fig.add_trace(go.Scatter(x=predicted_prices.date,
                         y=y_test,
                         name='Truth',
                         marker_color='LightSkyBlue',
                         showlegend=False), row=2, col=1)

fig.add_trace(go.Scatter(x=predicted_prices.date,
                         y=y_pred,
                         name='Prediction',
                         marker_color='MediumPurple',
                         showlegend=False), row=2, col=1)

fig.show()